In [0]:
%scala
println()
println()
println("Note: You can ignore the message 'Warning: classes defined within packages cannot be redefined without a cluster restart.'")

Note: You can ignore the message 'Warning: classes defined within packages cannot be redefined without a cluster restart.'

In [0]:
%scala
package com.databricks.training
object ClassroomHelper {
  def test_connection(url : String) : Unit = {
    val conn = java.sql.DriverManager.getConnection(url)
    try {
      val ps = conn.prepareStatement("SELECT 1")
      val rs = ps.executeQuery()
    } finally {
      conn.close()
    }
  }

  def sql_query(spark: org.apache.spark.sql.SparkSession, url : String, sql : String) : org.apache.spark.sql.DataFrame = {
    return spark.read.jdbc(url, s"($sql) query", new java.util.Properties)
  }

  def sql_update(url : String, sql : String, args : Any*) : Unit = {
    val conn = java.sql.DriverManager.getConnection(url)
    try {
      val ps = conn.prepareStatement(sql)
      args.zipWithIndex.foreach {case (arg, i) => {
        ps.setObject(i+1, arg)
      }}
      ps.executeUpdate()
    } finally {
      conn.close()
    }
  }

  def sql_update(url : String, sql : String, args : java.util.ArrayList[Object]) : Unit = {
    import scala.collection.JavaConverters._
    return sql_update(url, sql, args.asScala:_*)
  }

  def sql(url : String, sql : String) : Unit = {
    val conn = java.sql.DriverManager.getConnection(url)
    val stmt = conn.createStatement()
    val cmds = sql.split(";")
    var count = 0;
    try {
      for (cmd <- cmds) {
        if (!cmd.trim().isEmpty()) {
          stmt.addBatch(cmd)
          count += 1
        }
      }
      stmt.executeBatch()
    } finally {
      conn.close()
    }
  }
}

Warning: classes defined within packages cannot be redefined without a cluster restart.
Compilation successful.

In [0]:
%scala
import com.databricks.training.ClassroomHelper
spark.conf.set(
  "fs.azure.account.key.spearfishtrainingstorage.blob.core.windows.net",
  "cJoNObBZt8C8xz2GwQmaHx25DmyRXAyd8TEp7/HDlT6jgt4+LeOjwYEhQ5SsCCrO0HRy6xlL8WZEM6xEwE9+9Q==")
val blobStoreBaseURL = "wasbs://training-container-clean@spearfishtrainingstorage.blob.core.windows.net/"
val username = com.databricks.logging.AttributionContext.current.tags(com.databricks.logging.BaseTagDefinitions.TAG_USER);
val userhome = s"dbfs:/user/$username/"
spark.conf.set("spark.databricks.delta.preview.enabled", true)
spark.conf.set("com.databricks.training.username", username)
spark.conf.set("com.databricks.training.userhome", userhome)
displayHTML("") //Supress Output

In [0]:
%python
class ClassroomHelper(object):
  scalaHelper=spark._jvm.__getattr__("com.databricks.training.ClassroomHelper$").__getattr__("MODULE$")
  @classmethod
  def test_connection(cls, url):
    cls.scalaHelper.test_connection(url)
  @classmethod
  def sql_query(cls, spark, url, sql):
    return spark.read.jdbc(url, "({}) query".format(sql))
  @classmethod
  def sql_update(cls, url, sql, *args):
    cls.scalaHelper.sql_update(url, sql, args)
  @classmethod
  def sql(cls, url, sql):
    cls.scalaHelper.sql(url, sql)

blobStoreBaseURL = "wasbs://training-container-clean@spearfishtrainingstorage.blob.core.windows.net/"
username = spark.conf.get("com.databricks.training.username")
userhome = spark.conf.get("com.databricks.training.userhome")
None #Suppress output

In [0]:
%scala
println("Database-Setup successful.")

Database-Setup successful.